# Notebook 3 : SQL

In [ ]:
# Décommenter la ligne suivante pour installer ibis
# %pip install 'ibis-framework[sqlite]'

In [ ]:
import sqlite3

import pandas as pd
import ibis

from ibis import _

ibis.options.interactive = True

query_tables = "SELECT name FROM sqlite_master WHERE type='table'"

## STAR

Nous considérons les données des stations de vélos en libre service [STAR](https://www.star.fr/) de Rennes Métropole. Une copie de la base SQLite est disponible dans le fichier `star.db`. Nous utilisons d'abord Pandas pour répondre aux questions, puis Ibis.

1. Se connecter à la base de données et afficher la liste des tables à l'aide de la fonction `read_sql` de Pandas et de la requête `query_tables`.

2. Récupérer le contenu de la table `Etat` dans un dataframe et afficher la liste des variables disponibles. Même question pour la table `Topologie`.

3. Sélectionner l'identifiant `id`, le nom `nom` et l'identifiant de la station la plus proche `id_proche_1` depuis la table `Topologie`.

4. Faire une jointure sur la table précédente pour créer une table qui contient la liste des stations avec l'identifiant, le nom et le nom de la station la plus proche associée à l'identifiant `id_proche_1`. Les variables utilisées comme clés sont différents, penser à utiliser les arguments `left_on` et `right_on` de la méthode `merge`.

5. Ajouter à la table précédente la distance entre la station et la station la plus proche.

6. Créer une table avec le nom des trois stations les plus proches du point GPS *(48.1179151,-1.7028661)* classées par ordre de distance et le nombre de vélos disponibles dans ces stations.

7. Reprendre les questions précédentes en utilisant le module `ibis`. Pour les jointures, utiliser la méthode `left_join`.

8. (*Bonus*) Écrire des requêtes SQL pour obtenir les résultats demandés dans les questions 3 à 6. La fonction `to_sql` pourra être utilisée pour de l'aide.

## Musique

Le dépôt GitHub [lerocha/chinook-database](https://github.com/lerocha/chinook-database) met à disposition des bases de données de bibliothèques musicales. Une copie de la base SQLite est disponible dans le fichier `chinook.db`.

1. Utiliser le module `ibis` pour vous connecter à la base de données et explorer les tables formant le jeu de données pour le découvrir. En particulier, remarquer comment les tables `Playlist`, `PlaylistTrack` et `Track` sont liées entre elles.

2. Quelles sont les playlists qui contiennent le plus de pistes ?

3. Construire une table contenant les informations suivantes sur la playlist `Classical` : le titre de chaque piste ainsi que le titre de l'album dont cette piste est tirée.

4. (*Bonus*) Écrire une requête SQL donnant le résultat de la question précédente. La fonction `to_sql` pourra être utilisée pour de l'aide.